In [ ]:
%gui wx

%load_ext autoreload
%autoreload 2
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/Data"
#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import nilearn
#from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
#from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report # not all used


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

import pandas as pd

In [ ]:
current_dir = os.path.abspath("")
#print(f"current_dir: {current_dir}")
sys.path.append(current_dir)

dataset_id = 'ds000171'
subjects = ['sub-control{:02d}'.format(i+1) for i in range(20)]

dataset_path = os.path.join(current_dir, "data", dataset_id)
deriv_path = os.path.join(current_dir,"data", "derivatives")
preproc_path = os.path.join(deriv_path, 'preprocessed_data')

subject = "sub-control01"

mkdir_no_exist(dataset_path)
mkdir_no_exist(preproc_path)

In [ ]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

### Overview of the brain before any preprocessing

In [17]:
anatomical_path = op.join(dataset_path, subject, 'anat', '{}_T1w.nii.gz').format(subject)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(anatomical_path)

# 1. Skull Removal and fast tissue segmentation

In [5]:
from preprocessed import get_skull_stripped_anatomical

resulting_mask = get_skull_stripped_anatomical(dataset_path, preproc_path, subject, robust=True)

Done with BET.


In [ ]:
fsleyesDisplay.resetOverlays()

fsleyesDisplay.load(op.join(dataset_path, subject, 'anat', '{}_T1w.nii.gz').format(subject))
fsleyesDisplay.load(resulting_mask)

In [6]:
# The brain without skull is in the derivatives folder
from preprocessed import apply_fsl_mask

betted_brain_path = apply_fsl_mask(dataset_path, resulting_mask, preproc_path, subject)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

In [ ]:
#TODO discuss segmentation, not done because files not available - field map unwarping

In [7]:
from preprocessed import apply_fast
segmentation_path = apply_fast(preproc_path, subject)

Done with fast


In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_path, subject, 'anat','*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_path, subject, 'anat','*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_path, subject, 'anat','*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## Linear normalization using Ants
Using advanced normalization tools (ANTS), we standardize the fMRI to a standard, to be able to do comparisons.

In [8]:
type_of_transform = 'SyN'
subfolder='anat'
target = op.join(preproc_path, '{}'.format(subject), 'anat', '{}_T1w'.format(subject))
reference = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
moving_image = ants.image_read(target + '.nii.gz')
fixed_image = ants.image_read(reference + '.nii.gz')

    # Compute the transformation (non linear) to align the moving image to the fixed image
transformation = ants.registration(fixed=fixed_image, moving=moving_image, type_of_transform = type_of_transform)

    # After the transformation has been computed, apply it
warpedImage = ants.apply_transforms(fixed=fixed_image, moving=moving_image, transformlist=transformation['fwdtransforms'])

    # Save the image to disk
resultAnts = op.join(preproc_path, subject, subfolder, '{}_T1w_mni_{}.nii.gz'.format(subject, type_of_transform))
ants.image_write(warpedImage, resultAnts)

In [10]:
mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))

In [9]:
%autoreload 3

In [ ]:
from preprocessed import apply_ants
mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
resultAnts = apply_ants(preproc_path, subject, mni_template)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(mni_template)
fsleyesDisplay.load(resultAnts)

## Field Stabilisation
Field Stabilisation doesn't seem necessary

In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib

plt.plot(nib.load("/data/data/ds000171/sub-control01/func/sub-control01_task-music_run-1_bold.nii.gz").get_fdata().mean(axis=(0,1,2)))
plt.xlabel('Time (volume)')
plt.ylabel('Mean voxel intensity')

# fMRI Preprocessing

In [87]:
#standardize the runs and concatenate them together
runs = []
bids_root = dataset_path
for i in range(3):
    run = os.path.join(bids_root, subject, 'func', '{}_task-{}_run-{}_bold'.format(subject, task, i+1))
    mean = run.replace('ds000171', 'derivatives/preprocessed_data') + '_mean' 
    std = run.replace('ds000171', 'derivatives/preprocessed_data') + '_std'
    norm = run.replace('ds000171', 'derivatives/preprocessed_data') + '_norm'
    subprocess.run(['fslmaths', run, '-Tmean', mean])
    subprocess.run(['fslmaths', run, '-Tstd', std])
    subprocess.run(['fslmaths', run, '-sub', mean, '-div', std, norm])
    os.system('rm -rf {}'.format(op.join(preproc_path, subject, 'func', '*_mean*')))
    os.system('rm -rf {}'.format(op.join(preproc_path, subject, 'func', '*_std*')))
    runs.append(norm)
    
all_runs = os.path.join(preproc_path, subject, 'func', '{}_task-{}_run-{}_bold'.format(subject, task, 'all'))
subprocess.run(['fslmerge', '-t', all_runs, runs[0], runs[1], runs[2]])
os.system('rm -rf {}'.format(op.join(preproc_path, subject, 'func', '*_norm*')))

## Motion correction

In [ ]:
from preprocessed import apply_mcflirt
task = 'music'
run = 'all'
path_moco_data, reference_moco = apply_mcflirt(preproc_path, preproc_path, subject, task, run) # twice preproc_path since we start from 'all'
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(path_moco_data)

## Epi to anatomical coregistration

In [ ]:
# TODOOOOO reference_epi should be done on the moco one, ??? serie3 

Improvements forum: I usually use mcflirt with the  -meanvol option and then use the mean functional volume as input to  epi_reg.

In [50]:
from preprocessed import apply_epi_reg
epi_reg_path, reference_epi = apply_epi_reg(dataset_path, preproc_path, path_moco_data, subject, task, run)
# We did the coregistration of the reference volume and get a transform, now we need to apply it to all volumes

FLIRT pre-alignment
Running BBR
0.297502 0.999221 -0.039210 -0.004466 0.000000 0.039258 0.999165 0.011356 0.000000 0.004017 -0.011523 0.999925 0.000000 -5.417383 9.646792 -0.993922 1.000000 
Done with EPI to anatomical registration


In [51]:
# Inspect here if transformations worked
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)
fsleyesDisplay.load(epi_reg_path)

Now, how do we *know* if the registration is good or bad?
Well, there are several things to watch out for, but here are some main leads:
- Is the functional in the right orientation?
- Are the ventricles correctly aligned?
- Are the boundaries of the EPI more or less matching the anatomical?

➡️ You can also check how the white matter of the EPI matches your anatomical's white matter provided you have sufficient resolution

Note: __fast__ files are there for vizualisation purposes (they delete them)

Please ensure that:
- [ ] Applying ONLY motion correction transformation to the first volume yields the expected alignement (so it should be aligned with the \_moco volume.)
- [ ] Applying motion correction + epi -> anat should be aligned to anatomical
- [ ] Finally, applying motion correction + epi > anat + anat > standard should be aligned to the standard

Applying the transformation to a single volume is nice, but we should still need to know where the transformation was saved, to apply it to all other volumes of interest.

In [55]:
path_epi_transform = op.join(preproc_path, subject, 'func/sub-control01_task-music_run-1_bold_anat-space_epi.mat')

In [63]:
os.system('applyxfm4D') #Usage: applyxfm4D <input volume> <ref volume> <output volume> <transformation matrix file/dir>

Usage: /opt/fsl-6.0.7.4/bin/applyxfm4D <input volume> <ref volume> <output volume> <transformation matrix file/dir>

	--interp, -interp <nearestneighbour (or nn), trilinear, spline, sinc (default)>
	--singlematrix, -singlematrix (flag option, do not provide an argument)
	--fourdigit, -fourdigit (flag option, do not provide an argument)
	--userprefix, -userprefix <prefix>



65280

In [ ]:
# apply the transformation to all volumes
all_epi = epi_reg_path+'_4d'
#This should work with one transform for all volumes but apparently he wants one mat file per volume
#result = subprocess.run(['applyxfm4D', path_moco_data, anatomical_path, all_epi,path_epi_transform]) #, '-userprefix', ''

#Alternative that does not work neither
#subprocess.run(['flirt', '-in', path_moco_data, '-ref', anatomical_path,'-out', all_epi, '-init', path_epi_transform,'-applyxfm'])

In [70]:
# We split the volumes and apply it one by one
split_target = path_moco_data
splits_path = op.join(preproc_path, subject, 'func','splits')
mkdir_no_exist(splits_path)
split_name = op.join(splits_path, 'sub-control01_task-music_run-1_bold_split')

subprocess.run(['fslsplit', split_target, split_name, '-t'])

CompletedProcess(args=['fslsplit', '/data/data/derivatives/preprocessed_data/sub-control01/func/sub-control01_task-music_run-1_bold_moco', '/data/data/derivatives/preprocessed_data/sub-control01/func/splits/sub-control01_task-music_run-1_bold_split', '-t'], returncode=0)

In [77]:
split_vols = sorted(glob.glob(op.join(splits_path, '*_bold_split*')))
for i,split_vol in enumerate(split_vols): 
    split_nbr = split_vol.split('_')[-1].split('.')[0].split('split')[1]
    out_vol= op.join(preproc_path, subject, 'func', 'splits_epi','sub-control01_task-music_run-1_bold_epi_vol' + split_nbr)
    
    subprocess.run(['flirt', '-in', split_vol, '-ref', anatomical_path,'-out', out_vol,
                        '-init', path_epi_transform,'-applyxfm'])

In [68]:
os.system('rm -rf {}'.format(op.join(preproc_path, subject, 'func', '*_bold_split*')))

0

## For looking up stuff only, don't run

In [ ]:
from fsl.wrappers import applywarp

# We show this one when selecting the first EPI (volume 0000) 
target_epi = op.join(preproc_path, subject, 'func', '{}_task-sitrep_run-{}_bold_middle-vol'.format(subject, run))
split_nbr = '0000'
epi_moco = op.join(preproc_path, subject, 'func', subject+ '_task-music_run-1_bold_moco.mat/', 'MAT_' + split_nbr)

# We will name its warp as split0000
warp_name = op.join(preproc_path, subject, 'func', subject + '_split' + split_nbr + '_epi_2_std_warp')

# Get the transformation matrix of this volume (this one is actually the unit matrix, 
# since this volume is the reference)
#TODOOOOOOOOOOOOOOOO

# -- Step 1: Combine the transformations, that is :
#    EPI -> Motion correction -> Coregistration to anatomical -> Normalization to standard
#    EPI -> Motion correction is given by the matrix in sub-001_task-sitrep_run-01_bold_moco.mat/MAT_{vol_nbr}, where {vol_nbr} is the volume number of the volume of interest
#    EPI -> Coreg to anatomical, this is the _warp.nii.gz file in func/ folder


#    Anatomical > Template is saved by flirt when doing the anatomical to template coregistration, in anat/ folder
func_2_anat= op.join(preproc_path, subject, 'func', subject+ '_task-music_run-1_bold_anat-space_warp.nii.gz') # contains affine transformation from EPI to T1 sapce

combine_all_transforms(ref, warp_name, True, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)
out_vol= op.join(preproc_path, subject, 'func', subject+ '_task-music_run-1_bold_std_vol' + split_nbr)

# -- Step 2: Apply the transformation to our EPI
subprocess.run(['applywarp', '-i', target_epi, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs', '--premat={}'.format(epi_moco)])

### 1.2.4 Applying the transformation to the entire timeseries at last - flirt does this on its own

In [ ]:
split_target = original_epi
split_name = op.join(preproc_path, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_split')

subprocess.run(['fslsplit', split_target, split_name, '-t'])
print_dir_tree(bids_root,max_depth=5)

In [ ]:
# Let's combine the different transforms EXCEPT motion correction!
warp_name = op.join(preproc_path, 'sub-001', 'func', 'sub-001_epi_moco_2_std_warp')

print("Starting to combine transforms...")
#combine_all_transforms(ref, warp_name,  True, epi_2_moco=None, epi_2_anat_warp=func_2_anat, anat_2_standard_warp=anat_2_mni_trans)
print("Done, moving on to application of transforms...")

###########
# Now apply transformation to all our volumes.
# We will remember the volumes as well, to group them back afterwards.
##########

# Notice that we are sorting the volumes here! This is important, to make sure we don't get them in random order :)
split_vols = sorted(glob.glob(op.join(preproc_path, subject, 'func', '*_bold_split*')))


# Define a function that wraps subprocess.run()
def run_subprocess(split_vol, vol_nbr):
    """
    SAFETY GOGGLES ON
    This function launches applywarp in parallel to reach complete result quicker

    Parameters
    -----------
    split_vol: str
        Path to the volume on which to apply the transformation
    vol_nbr: str
        Number of the volume in the timeserie. Useful to reorder volumes after the fact, since parallelisation does not honour order.

    Returns
    -------
    out_vol: str
        Path to the transformed volume
    vol_nbr: str
        Number of the volume in the timeserie. Useful to reorder volumes after the fact.
    """
    try:
        split_nbr = split_vol.split('_')[-1].split('.')[0].split('split')[1]
        epi_moco = op.join(preproc_path, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_moco.mat/', 'MAT_' + split_nbr)
        out_vol= op.join(preproc_path, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_std_vol' + split_nbr)
        result = subprocess.run(['applywarp', '-i', split_vol, '-r', ref, '-o', out_vol, '-w', warp_name, '--abs', '--premat={}'.format(epi_moco)], check=True)
        return out_vol, vol_nbr
    except subprocess.CalledProcessError as e:
        return f"applywarp for volume '{split_vol}' failed with error: {e.stderr.decode('utf-8')}"


produced_vols = [None]*len(split_vols)
# Initialize ThreadPoolExecutor and the progress bar
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Use tqdm to wrap the futures
    with tqdm(total=len(split_vols)) as progress:
        # Launch subprocesses in parallel
        futures = {executor.submit(run_subprocess, vol,i): vol for i,vol in enumerate(split_vols)}

        # Process completed tasks
        for future in concurrent.futures.as_completed(futures):
            out_vol, vol_nbr = future.result()  # Get the result of the subprocess
            produced_vols[vol_nbr] = out_vol
            # Update the progress bar for each completed task
            progress.update(1)

## Smoothing

In [97]:
mkdir_no_exist(op.join(preproc_path, subject, 'func', 'final'))
output_path = op.join(preproc_path, subject, 'func', 'final','sub-control01_task-music_run-all_bold') 
subprocess.run(['fslmaths',path_moco_data, '-s', str(6/2.3548), '{}_smoothed-6mm'.format(output_path)])

CompletedProcess(args=['fslmaths', '/data/data/derivatives/preprocessed_data/sub-control01/func/sub-control01_task-music_run-all_bold_moco', '-s', '2.547987090198743', '/data/data/derivatives/preprocessed_data/sub-control01/func/final/sub-control01_task-music_run-all_bold_smoothed-6mm'], returncode=0)

## Slice Time correction (a faire encore)

In [ ]:
#Very big red box telling you that this part takes 1h

In [ ]:
import pandas as pd

In [ ]:
task='music'
data = pd.read_json(op.join(dataset_path, 'task-{}_bold.json'.format(task)), typ= 'series')
slice_timing = data['SliceTiming']
tr = data['RepetitionTime'] 

In [ ]:
# get the informations in the header -to determine the number of slices
os.system('fslhd {}'.format(op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold.nii.gz')))

In [ ]:
len(slice_timing) #check which dimensions has the slices

In [ ]:
slice_order = np.argsort(slice_timing) + 1

# Write to a file the corresponding sorted timings :)
timing_path = op.join(preproc_path,  'sub-001', 'func', 'sub-001_task-sitrep_run-01_slice-timings.txt')
file = open(timing_path, mode='w')
for t in slice_order:
    file.write(str(t) + '\n')
file.close()

In [ ]:
file_to_realign = op.join(bids_root, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold')
output_target = op.join(preproc_path, 'sub-001', 'func', 'sub-001_task-sitrep_run-01_bold_slice-corr')

subprocess.run(['slicetimer', '-i', file_to_realign, '-o', output_target, '-r', str(tr), '-d', str(3), '--ocustom={}'.format(timing_path)])
